<a href="https://colab.research.google.com/github/FranciscoBPereira/DeepLearning-UBologna/blob/main/UBologna24_Ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 8)


# TensorFlow ≥2.10 is required
import tensorflow as tf
assert tf.__version__ >= "2.10"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
# Load CIFAR10 dataset from keras datasets:
# https://keras.io/api/datasets/cifar10/
# https://www.cs.toronto.edu/~kriz/cifar.html

# This is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 10 categories.

# The load_data() method creates train and test sets.

from keras.datasets import cifar10

(train_images_full, train_labels_full), (test_images, test_labels) = cifar10.load_data()

train_labels_full = train_labels_full.squeeze()
test_labels = test_labels.squeeze()

# Images are not standardized. They will be preprocessed by the model

# We further divide the original train datasets into train and validation datasets
train_images = train_images_full[5000:]
valid_images = train_images_full[:5000]
train_labels = train_labels_full[5000:]
valid_labels = train_labels_full[:5000]

In [ ]:
# Obtain the pretrained Xception Model from keras applications: https://keras.io/api/applications/
# It does not include the classification section (include_top = False) and weights are frozen

base_model = keras.applications.Xception(include_top=False, weights='imagenet', input_shape=(224,224,3))

base_model.trainable = False


In [ ]:

# The complete model: It comprises the Xception model and the classification layers
# The Global Average Pooling layer is used to linearize information

# In this example, the Keras functional API is used

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

inputs = keras.Input(shape=(32, 32, 3))

x = layers.Lambda(lambda image: tf.image.resize(image, (224,224)))(inputs)
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(10, activation=('softmax'))(x)

model = tf.keras.Model(inputs, outputs)



In [ ]:
# Present a summary of the network architecture
# Confirm the architecture and the number of parameters

model.summary()

In [ ]:
# A NN has to be compiled by calling the compile() method: https://keras.io/api/models/model_training_apis/
# Three components have to be defined (recall how backpropagation is appplied during training):
# 1. the Optimizer to be used in training
# 2. The loss function
# 3. The evaluation metric

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

model.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
# Training is performed by calling the fit() method: https://keras.io/api/models/model_training_apis/

# Hyper-parameters: batch size and epochs
# Validation datasets can also be provided

# It returns a history object.
# Its History.history attribute is a record of training loss values and metrics values at successive epochs,
# as well as validation loss values and validation metrics values (if applicable).

# Only 5 epochs are performed, as training will take a long time

history = model.fit(train_images, train_labels, batch_size=32, epochs=5,
                    validation_data=(valid_images, valid_labels))


In [ ]:
# Plot the evolution of the accuracy metrics

import pandas as pd

x = pd.DataFrame(history.history, columns = ['accuracy', 'val_accuracy'])
x.plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
# Evaluation the generalization ability of the model
# The test set will be used in this step
# Classification of a set of examples can be performed using the evaluate() method:  https://keras.io/api/models/model_training_apis/

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test Accuracy: ', test_acc)

In [ ]:
dot_img_file = '/tmp/model_3.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)